In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizontal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,178996.06,-1.11,-0.12,0.57,-0.01,-0.07,0.01,-0.00,-0.00
3,-1.11,37348.96,-0.20,0.08,-0.08,0.01,0.02,0.00,0.00
4,-0.12,-0.20,7630.00,0.11,-0.00,-0.00,-0.01,-0.00,0.00
5,0.57,0.08,0.11,1531.46,0.00,-0.01,0.01,0.00,0.00
6,-0.01,-0.08,-0.00,0.00,286.98,-0.00,-0.00,0.00,0.00
7,-0.07,0.01,-0.00,-0.01,-0.00,50.56,0.00,0.00,0.00
8,0.01,0.02,-0.01,0.01,-0.00,0.00,8.41,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,1.29,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.15


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00003,-0.00000,-0.00002,0.00001,-0.00001,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00001,0.00004,0.00000,0.00001
4,-0.00000,-0.00001,1.00000,0.00003,-0.00000,-0.00000,-0.00003,-0.00000,0.00001
5,0.00003,0.00001,0.00003,1.00000,0.00001,-0.00002,0.00005,0.00000,0.00001
6,-0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00002,0.00000
7,-0.00002,0.00001,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00000
8,0.00001,0.00004,-0.00003,0.00005,-0.00001,0.00002,1.00000,0.00003,0.00000
9,-0.00001,0.00000,-0.00000,0.00000,0.00002,0.00000,0.00003,1.00000,-0.00000
10,-0.00000,0.00001,0.00001,0.00001,0.00000,0.00000,0.00000,-0.00000,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.8109813910611117

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.56259380e+09 1.52605935e+08 6.23930895e+06 2.44385516e+05
 8.24067900e+03 2.39963682e+02 5.91677857e+00 1.04600257e-01
 3.64297004e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,9.996143e-01,-0.027239,-0.005302,-0.001059,-0.000199,-0.000035,-0.000006,-9.178090e-07,-1.065580e-07
1,2.705795e-02,0.999148,-0.030572,-0.005852,-0.001097,-0.000192,-0.000031,-4.909772e-06,-5.621258e-07
2,6.082696e-03,0.030188,0.998893,-0.034973,-0.006260,-0.001101,-0.000184,-2.812705e-05,-3.236481e-06
3,1.418724e-03,0.006822,0.034496,0.998597,-0.038997,-0.006595,-0.001092,-1.677397e-04,-1.947423e-05
4,3.192956e-04,0.001533,0.007500,0.038401,0.998164,-0.045615,-0.007305,-1.116121e-03,-1.300236e-04
5,7.053821e-05,0.000336,0.001646,0.008213,0.044818,0.997287,-0.057156,-8.439866e-03,-9.816518e-04
6,1.545399e-05,0.000074,0.000365,0.001809,0.009678,0.055919,0.995273,-7.829903e-02,-8.801463e-03
7,3.559347e-06,0.000017,0.000083,0.000413,0.002205,0.012526,0.076139,9.901652e-01,-1.166780e-01
8,8.368661e-07,0.000004,0.000020,0.000097,0.000519,0.002947,0.017708,1.156272e-01,9.931303e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0003857003971744488,
 0.0008515072055060857,
 0.0011065274863282237,
 0.001402970129909753,
 0.0018359994751150355,
 0.002712638575736337,
 0.004727268955913466,
 0.009834812210887667,
 0.006869697142005915]